In [8]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Input')
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '012225'

library(missForest)
library(readxl)
library(tidyverse)
library(imputeLCMD)
library(factoextra)
# library(vegan)
# library(preprocessCore)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Woodsmoke_Data_012125.xlsx", sheet = 2))
wf_df = data.frame(Data = 'WF', read_excel("Wildfire_Data_012125.xlsx", sheet = 2))

In [9]:
head(ws_df)
head(wf_df)

,Data,HAWC.ID,Study,Replicate,Sample_Number,Chemical_Class,Metric,DTXSID,Name,Sample_Identifier,Value
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,429174,Arif et al. 2017,1,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,DTXSID1074759_PAH_1,NA
2,WS,429174,Arif et al. 2017,2,2,PAH,Weight,DTXSID1074759,1-Methylchrysene,DTXSID1074759_PAH_2,NA
3,WS,429174,Arif et al. 2017,1,3,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,DTXSID9020877_PAH_3,NA
4,WS,429174,Arif et al. 2017,2,4,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,DTXSID9020877_PAH_4,NA
5,WS,429174,Arif et al. 2017,1,5,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,DTXSID9020877_PAH_5,NA
6,WS,429174,Arif et al. 2017,2,6,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,DTXSID9020877_PAH_6,NA


,Data,HAWC.ID,Study,Sample_Number,Chemical_Class,Metric,DTXSID,Name,Sample_Identifier,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID6024254,Phenanthrene,DTXSID6024254_PAH_1,0.154
2,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,DTXSID1074759_PAH_1,NA
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,DTXSID9020877_PAH_1,NA
4,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID0029187,"2,6-Dimethylnaphthalene",DTXSID0029187_PAH_1,NA
5,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,DTXSID4020878_PAH_1,NA
6,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID3061134,9-Methylanthracene,DTXSID3061134_PAH_1,NA
